In [93]:
import tifffile
import math
import cv2
import numpy as np
import ComputeGaussianPSF as CG

import matplotlib.pyplot as plt

This code is for creating single phase substacks from 9 sim planes

In [94]:
filepath = '/home/imaging/Documents/MATLAB_SIM/OTF_SIMULATION/single point/7phase single point 3phase 144nm 26p5nm scaled/angle2/phase'
filepathOut = '/home/imaging/Documents/MATLAB_SIM/OTF_SIMULATION/single point/7phase single point 3phase 144nm 26p5nm scaled/'
filepathEnd = '/stacked_planes/'
# writepath = 'stacked_planes\\'
# phase = 0
# out_filename = 'lateral_stack_axial_phase' + str(phase) #don't include .tif suffix
# ph0 = tifffile.imread(filepath + str(0) + filepathEnd + 'phase' + str(phase) + '.tif')
# ph1 = tifffile.imread(filepath + str(1) + filepathEnd + 'phase' + str(phase) + '.tif')
# ph2 = tifffile.imread(filepath + str(2) + filepathEnd + 'phase' + str(phase) + '.tif')
# ph3 = tifffile.imread(filepath + str(3) + filepathEnd + 'phase' + str(phase) + '.tif')
# ph4 = tifffile.imread(filepath + str(4) + filepathEnd + 'phase' + str(phase) + '.tif')
phases = 7
phase = 0
out_filename = 'lateral_stack_axial_phase' + '_angle2' #don't include .tif suffix
ph0 = tifffile.imread(filepath + str(0) + filepathEnd + 'focus_subStack_0' + '.tif')
ph1 = tifffile.imread(filepath + str(1) + filepathEnd + 'focus_subStack_0' + '.tif')
ph2 = tifffile.imread(filepath + str(2) + filepathEnd + 'focus_subStack_0' + '.tif')
ph3 = tifffile.imread(filepath + str(3) + filepathEnd + 'focus_subStack_0' + '.tif')
ph4 = tifffile.imread(filepath + str(4) + filepathEnd + 'focus_subStack_0' + '.tif')
if phases > 5:
    ph5 = tifffile.imread(filepath + str(5) + filepathEnd + 'focus_subStack_0' + '.tif')
    ph6 = tifffile.imread(filepath + str(6) + filepathEnd + 'focus_subStack_0' + '.tif')


IS_SIM = True


In [95]:

print(ph0.shape)
print(ph0.dtype)
dims = ph0.shape
print(dims[0])
print(dims[1])
print(dims[2])

reverse_stack = False

a = np.zeros((dims[0]*phases, dims[1], dims[2])).astype(ph0.dtype)
c = np.zeros((dims[0]*phases, dims[1], dims[2])).astype(ph0.dtype)

upper_z = dims[1]//2 + 1 - dims[0]//2
lower_z = dims[1]//2 + 1 + dims[0]//2

for i in range(dims[0]):
    a[i*phases,:,:] = ph0[i,:,:]
    a[i*phases + 1,:,:] = ph1[i,:,:]
    a[i*phases + 2,:,:] = ph2[i,:,:]
    a[i*phases + 3,:,:] = ph3[i,:,:]
    a[i*phases + 4,:,:] = ph4[i,:,:]
    if phases > 5:
        a[i*phases + 5,:,:] = ph5[i,:,:]
        a[i*phases + 6,:,:] = ph6[i,:,:]



if reverse_stack and phases == 7:    
    for i in range(dims[0]):
        c[i*phases,:,:] = ph6[i,:,:]
        c[i*phases + 1,:,:] = ph5[i,:,:]
        c[i*phases + 2,:,:] = ph4[i,:,:]
        c[i*phases + 3,:,:] = ph3[i,:,:]
        c[i*phases + 4,:,:] = ph2[i,:,:]
        if phases > 5:
            c[i*phases + 5,:,:] = ph1[i,:,:]
            c[i*phases + 6,:,:] = ph0[i,:,:]
elif reverse_stack and phases < 7:
    for i in range(dims[0]):
        c[i*phases,:,:] = ph4[i,:,:]
        c[i*phases + 1,:,:] = ph3[i,:,:]
        c[i*phases + 2,:,:] = ph2[i,:,:]
        c[i*phases + 3,:,:] = ph1[i,:,:]
        c[i*phases + 4,:,:] = ph0[i,:,:]

    
#     a[3,:,:] = x4[i,:,:]
#     a[4,:,:] = x5[i,:,:]
#     a[5,:,:] = x6[i,:,:]
#     a[6,:,:] = x7[i,:,:]
#     a[7,:,:] = x8[i,:,:]
#     a[8,:,:] = x9[i,:,:]
scale = True
dims = a.shape
if scale:
    b = np.zeros((dims[0], np.ceil(dims[1]/2).astype(np.uint16), np.ceil(dims[2]/2).astype(np.uint16))).astype(a.dtype) 
    for i in range(dims[0]):
        b[i,:,:] = cv2.resize(a[i,:,:], dsize=(np.ceil(dims[1]/2).astype(np.uint16), np.ceil(dims[2]/2).astype(np.uint16)), interpolation=cv2.INTER_CUBIC)
        maxb = np.max(b)
    if reverse_stack:
        d = np.zeros((dims[0], np.ceil(dims[1]/2).astype(np.uint16), np.ceil(dims[2]/2).astype(np.uint16))).astype(a.dtype) 
        for i in range(dims[0]):
            d[i,:,:] = cv2.resize(c[i,:,:], dsize=(np.ceil(dims[1]/2).astype(np.uint16), np.ceil(dims[2]/2).astype(np.uint16)), interpolation=cv2.INTER_CUBIC)
            maxb = np.max(d)
        tifffile.imwrite(filepathOut + out_filename + '_reverse.tif', d)
    # b = ((b/maxb)*(2**16-1)).astype(np.uint16) #convert to 16 bit    
    # b = b.astype(np.uint16)   
    tifffile.imwrite(filepathOut + out_filename + '.tif', b)
else:
    tifffile.imwrite(filepathOut + out_filename + '.tif', a)
    if reverse_stack:
         tifffile.imwrite(filepathOut + out_filename + '_reverse.tif', c)



# frame_div = 1



# if IS_SIM:
#     frame_div = 15

# a = np.zeros((9*frame_div,dims[2],dims[1])).astype(np.uint16)

# for i in range(dims[0]//frame_div):
#     for j in range(frame_div):
#         a[0*frame_div+j,:,:] = BL[i*frame_div + j,:,:]
#         a[1*frame_div+j,:,:] = BM[i*frame_div + j,:,:]
#         a[2*frame_div+j,:,:] = BR[i*frame_div + j,:,:]
#         a[3*frame_div+j,:,:] = ML[i*frame_div + j,:,:]
#         a[4*frame_div+j,:,:] = MM[i*frame_div + j,:,:]
#         a[5*frame_div+j,:,:] = MR[i*frame_div + j,:,:]
#         a[6*frame_div+j,:,:] = TL[i*frame_div + j,:,:]
#         a[7*frame_div+j,:,:] = TM[i*frame_div + j,:,:]
#         a[8*frame_div+j,:,:] = TR[i*frame_div + j,:,:]
    
#     tifffile.imwrite(filepath + writepath + 'stack' + str(i) + '.tif', a)


(27, 513, 513)
float32
27
513
513
